In [1]:
#import dependencies
import pandas as pd
import json
import os

In [2]:
jsondata = os.path.join("Resources","purchase_data.json")

In [3]:
#Read JSON data into a variable
with open(jsondata) as json_data:
    d = json.load(json_data)

In [4]:
#turn data into dataframe
game_df = pd.DataFrame(d, columns=['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
game_df.head()

,SN,Age,Gender,Item ID,Item Name,Price
0,Aelalis34,38,Male,165,Bone Crushing Silver Skewer,3.37
1,Eolo46,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Assastnya25,34,Male,174,Primitive Blade,2.46
3,Pheusrical25,21,Male,92,Final Critic,1.36
4,Aela59,23,Male,63,Stormfury Mace,1.27


In [5]:
#Player Count
total_players = len(game_df['SN'].value_counts())
Total_Players = pd.DataFrame({"Total Players": total_players}, index=[0])
Total_Players

,Total Players
0,573


In [6]:
#Purchasing Analysis

In [35]:
#Number of Unique Items
unique_items = len(game_df['Item ID'].value_counts())

#Average Purchase Price
average_price = game_df['Price'].mean()

#Total Number of Purchases
total_purchases = game_df['Item Name'].count()


#Total Revenue
total_revenue = game_df['Price'].sum()

#Create DataFrame
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [unique_items],
                                   "Average Price": [average_price],
                                   "Total Purchases": [total_purchases],
                                   "Total Revenue": [total_revenue],
                                
})

#Reorder DataFrame
purchasing_analysis = purchasing_analysis[["Number of Unique Items", "Average Price","Total Purchases", "Total Revenue"]]
                                

#improve formatting
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${0:,.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${0:,.2f}".format)

#Reorder Columns
purchasing_analysis

,Number of Unique Items,Average Price,Total Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [8]:
#Gender Demographics

In [9]:
#Group data by Gender and filter duplicates
grouped_df = game_df.groupby(["Gender"])
unique_df = grouped_df.nunique()

#Total Gender
total_gender = unique_df["SN"].sum()

#Percentage and Count of Players
count = unique_df["SN"].unique()
percentage = unique_df["SN"]/ total_gender

#Create new dataframe
final_gender = pd.DataFrame({"Percentage of Players": percentage,
                            "Count":count})
#Change percentage format and re order columns
final_gender["Percentage of Players"] = final_gender["Percentage of Players"].map("{:,.2%}".format) 
final_gender.columns = ["Percentage of Players", "Count"]
#Print final dataframe
final_gender

,Percentage of Players,Count
Gender,,
Female,100,17.45%
Male,465,81.15%
Other / Non-Disclosed,8,1.40%


In [31]:
#Purchasing Analysis (Gender)

#Purchase Count
purchase_count = unique_df["Gender"].value_counts()

#Average Purchase Price
average_price = grouped_df["Price"].mean()

#Total Purchase Value
purchase_price = grouped_df["Price"].sum()

#Normalized Totals
normalized = purchase_price / count

#Create new dataframe
gender_analysis = pd.DataFrame({"Average Purchase Price": average_price,"Total Purchase Price":purchase_price,"Normalized Totals": normalized})

#Clean up formatting and reorder columns
gender_analysis["Average Purchase Price"] = gender_analysis["Average Purchase Price"].map("${:,.2f}".format) 
gender_analysis["Total Purchase Price"] = gender_analysis["Total Purchase Price"].map("${:,.2f}".format) 
gender_analysis["Normalized Totals"] = gender_analysis["Normalized Totals"].map("${:,.2f}".format) 
#Reorder Columns
gender_analysis = gender_analysis[["Average Purchase Price", "Total Purchase Price", "Normalized Totals"]]
gender_analysis

,Average Purchase Price,Total Purchase Price,Normalized Totals
Gender,,,
Female,$2.82,$382.91,$3.83
Male,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,$3.25,$35.74,$4.47


In [32]:
#Age Demographics
#Drop Duplicates
cleaned_df = game_df.drop_duplicates("SN")

#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
bins = [10, 14, 18, 22, 26, 30, 34, 38, 42]
groups = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Create a  new column for age groups and then groupby Age Groups
cleaned_df["Age Groups"] = pd.cut(cleaned_df["Age"], bins, labels=groups)
age_df = cleaned_df.groupby(["Age Groups"])

total_age = unique_df["Age"].sum()

#Purchase Count
age_purchase = cleaned_df["Age Groups"].value_counts()

#Percentage of Users
age_percentage = age_purchase / total_players

age_demographics = pd.DataFrame({"Total Count": age_purchase,
                             "Percentage of Players":age_percentage})

age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map("{:,.2%}".format) 

age_demographics = pd.concat([age_demographics.loc[["<10"],:], age_demographics.drop("<10", axis=0)], axis=0)

age_demographics

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Percentage of Players,Total Count
<10,3.49%,20
15-19,31.06%,178
20-24,26.70%,153
10-14,14.66%,84
25-29,7.68%,44
30-34,5.93%,34
35-39,4.36%,25
40+,1.92%,11


In [29]:
#Age Demographics

#Average Purchase Price
age_average_price = age_df["Price"].mean()

#Total Purchase Value
age_price = age_df["Price"].sum()

#Normalized Totals
normalized_age = age_price / age_purchase

#Create new dataframe
age_analysis = pd.DataFrame({"Purchase Count": age_purchase,
                             "Average Purchase Price":age_average_price,
                            "Total Purchase Value":age_price,
                            "Normalized Totals": normalized_age})

#Clean up formatting
age_analysis["Average Purchase Price"] = age_analysis["Average Purchase Price"].map("${:,.2f}".format) 
age_analysis["Total Purchase Value"] = age_analysis["Total Purchase Value"].map("${:,.2f}".format) 
age_analysis["Normalized Totals"] = age_analysis["Normalized Totals"].map("${:,.2f}".format) 

#Reorder Columns
age_analysis = age_analysis[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

#Move bottom row to the top
age_analysis = pd.concat([age_analysis.loc[["<10"],:], age_analysis.drop("<10", axis=0)], axis=0)

age_analysis


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
<10,20,$2.60,$51.99,$2.60
10-14,84,$2.89,$242.39,$2.89
15-19,178,$2.94,$523.43,$2.94
20-24,153,$3.01,$460.79,$3.01
25-29,44,$2.98,$131.23,$2.98
30-34,34,$3.35,$113.86,$3.35
35-39,25,$2.89,$72.23,$2.89
40+,11,$3.10,$34.14,$3.10


In [28]:
#Top Users
grouped_sn = game_df.groupby(["SN"])

#Find total spent per user
total_price_sn = grouped_sn.sum()["Price"]

#Find avg spent per user
avg_price_sn = grouped_sn.mean()["Price"]

#Find purchase count per user
count_sn = grouped_sn.count()["Price"]

#Create new dataframe
top_user_df = pd.DataFrame({"Purchase Count":count_sn,
                            "Average Purchase Price":avg_price_sn,
                            "Total Purchase Price": total_price_sn
                            })
#Sort by total purchase price

sorted_df = top_user_df.sort_values("Total Purchase Price",ascending=False)

#Format numbers
sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${:,.2f}".format) 
sorted_df["Total Purchase Price"] = sorted_df["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_df = sorted_df[["Purchase Count", "Average Purchase Price", "Total Purchase Price"]]

#Display top 5
sorted_df.head(5)


,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [25]:
#Most Popular Items
grouped_id = game_df.set_index(["Item ID", "Item Name"])

grouped_id = grouped_id.groupby(level=["Item ID", "Item Name"])

#Find total spent per user
total_price_id = grouped_id.sum()["Price"]

#Find avg spent per user
avg_price_id = grouped_id.mean()["Price"]

#Find purchase count per user
count_id = grouped_id.count()["Price"]


#Create new dataframe
items_df = pd.DataFrame({ 
                         "Count":count_id,
                            "Average Purchase Price":avg_price_id,
                            "Total Purchase Price": total_price_id,
                            })


#Sort by total purchase price
sorted_items = items_df.sort_values("Count",ascending=False)

sorted_items["Average Purchase Price"] = sorted_items["Average Purchase Price"].map("${:,.2f}".format) 
sorted_items["Total Purchase Price"] = sorted_items["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_items = sorted_items[["Count", "Average Purchase Price", "Total Purchase Price"]]


#Display top 5
sorted_items.head(5)



,,Count,Average Purchase Price,Total Purchase Price
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [26]:
#Most Profitable Items
grouped_id = game_df.set_index(["Item ID", "Item Name"])

grouped_id = grouped_id.groupby(level=["Item ID", "Item Name"])

#Find total spent per user
total_price_id = grouped_id.sum()["Price"]

#Find avg spent per user
avg_price_id = grouped_id.mean()["Price"]

#Find purchase count per user
count_id = grouped_id.count()["Price"]

#Create new dataframe
items_df = pd.DataFrame({ "Count":count_id,
                        "Average Purchase Price":avg_price_id,
                        "Total Purchase Price": total_price_id,
                            })


#Sort by total purchase price
sorted_items = items_df.sort_values("Total Purchase Price",ascending=False)

sorted_items["Average Purchase Price"] = sorted_items["Average Purchase Price"].map("${:,.2f}".format) 
sorted_items["Total Purchase Price"] = sorted_items["Total Purchase Price"].map("${:,.2f}".format) 

#Reorder Columns
sorted_items = sorted_items[["Count", "Average Purchase Price", "Total Purchase Price"]]

#Display top 5
sorted_items.head(5)


,,Count,Average Purchase Price,Total Purchase Price
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
